<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/RBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/adverserial_attacks_functions.py'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 4.35MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 2.61MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 2.27MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 13.5MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 8.47MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *


torch.manual_seed(0)



cuda
cuda


In [4]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [30]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [31]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [7]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [8]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
# Create an instance of your model
model_AT_rFGSM_weightedLoss = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM_weightedLoss.load_state_dict(torch.load('model_AT_rFGSM_weightedLoss.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [10]:
from sklearn.cluster import KMeans

# Function to initialize centers and sigmas separately for benign and malware samples using KMeans clustering
def initialize_centers_sigmas_separate(data_loader, num_centers_per_class):
    centers_benign = []
    centers_malware = []

    # Collect data into a single tensor
    all_data = torch.cat([batch for batch, _ in data_loader], dim=0)
    all_data = all_data.numpy()

    # Collect labels into a single tensor
    all_labels = torch.cat([labels for _, labels in data_loader], dim=0)
    all_labels = all_labels.numpy()

    # Separate benign and malware samples
    benign_indices = np.where(all_labels == 0)[0]
    malware_indices = np.where(all_labels == 1)[0]

    # Clustering for benign samples
    benigns = all_data[benign_indices]
    subset_benigns = benigns[:20000]
    kmeans_benign = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_benign.fit(subset_benigns)
    centers_selected_benign = kmeans_benign.cluster_centers_

    # Clustering for malware samples
    kmeans_malware = KMeans(n_clusters=num_centers_per_class, init='k-means++', n_init='auto')
    kmeans_malware.fit(all_data[malware_indices])
    centers_selected_malware = kmeans_malware.cluster_centers_

    # Combine selected centers
    all_centers = np.concatenate([centers_selected_benign, centers_selected_malware], axis=0)

    # Calculate sigma based on the average distance between centers
    total_distance = 0.0
    num_pairs = 0

    # Calculate pairwise distances between centers for benign samples
    for i in range(len(centers_selected_benign)):
        for j in range(i + 1, len(centers_selected_benign)):
            distance_ij = np.sqrt(((centers_selected_benign[i] - centers_selected_benign[j]) ** 2).sum())
            total_distance += distance_ij
            num_pairs += 1

    # Calculate pairwise distances between centers for malware samples
    for i in range(len(centers_selected_malware)):
        for j in range(i + 1, len(centers_selected_malware)):
            distance_ij = np.sqrt(((centers_selected_malware[i] - centers_selected_malware[j]) ** 2).sum())
            total_distance += distance_ij
            num_pairs += 1

    # Calculate mean sigma

    sigma = total_distance / num_pairs

    return all_centers, sigma


In [11]:
all_centers, sigma = initialize_centers_sigmas_separate(train_loader, 500)

In [12]:
sigma

6.325346210357184

In [13]:
all_centers.shape

(1000, 10000)

In [14]:
# Save the array to a file
np.save('all_centers.npy', all_centers)

In [11]:
# Load the array from the file
all_centers = np.load('all_centers.npy')

sigma = 6.281732838681898

In [12]:
# Move all_centers back to GPU
all_centers = torch.tensor(all_centers, device=device)

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from sklearn.cluster import KMeans

# Define the RBF model
class RBFModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, init_centers, init_sigmas):
        super(RBFModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.centers = nn.Parameter(torch.Tensor(init_centers))
        self.sigmas = nn.Parameter(torch.Tensor(init_sigmas))

        # Linear layer for output
        self.linear = nn.Linear(hidden_dim, output_dim)

    def gaussian(self, x, c, sigma):
        return torch.exp(-torch.sum((x[:, None, :] - c) ** 2, dim=-1) / (2 * sigma ** 2))

    def forward(self, x):
        radial_out = self.gaussian(x, self.centers, self.sigmas)
        output = self.linear(radial_out.to(torch.float32))
        return output

# Function to evaluate the model on a dataset
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(torch.float32).to(device), batch_y.to(device)  # Move data to GPU
            output = model(batch_x)
            loss = criterion(output, batch_y.squeeze().to(torch.long))
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == batch_y.squeeze()).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / len(data_loader.dataset)

    return avg_loss, accuracy


In [16]:
# Run this to release GPU memory
torch.cuda.empty_cache()

In [ ]:
# Instantiate the RBF model with initialized centers and sigmas
model = RBFModel(10000, 1000, 2, all_centers, [sigma])
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0

    for i, (batch_x, batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(torch.float32).to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y.squeeze().to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct_predictions += (predicted == batch_y.squeeze()).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss}, Train Accuracy: {accuracy}')

    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


Epoch [1/10], Step [10/897], Loss: 0.0172998309135437
Epoch [1/10], Step [20/897], Loss: 0.032817691564559937
Epoch [1/10], Step [30/897], Loss: 0.05968206748366356
Epoch [1/10], Step [40/897], Loss: 0.1018967255949974
Epoch [1/10], Step [50/897], Loss: 0.11222320050001144
Epoch [1/10], Step [60/897], Loss: 0.04457662254571915
Epoch [1/10], Step [70/897], Loss: 0.033884137868881226
Epoch [1/10], Step [80/897], Loss: 0.17707860469818115
Epoch [1/10], Step [90/897], Loss: 0.018819378688931465
Epoch [1/10], Step [100/897], Loss: 0.05857442319393158
Epoch [1/10], Step [110/897], Loss: 0.008322658017277718
Epoch [1/10], Step [120/897], Loss: 0.0827416181564331
Epoch [1/10], Step [130/897], Loss: 0.21955044567584991
Epoch [1/10], Step [140/897], Loss: 0.08162592351436615
Epoch [1/10], Step [150/897], Loss: 0.04201076924800873
Epoch [1/10], Step [160/897], Loss: 0.012118374928832054
Epoch [1/10], Step [170/897], Loss: 0.05060906335711479
Epoch [1/10], Step [180/897], Loss: 0.05158982798457146

In [27]:
    # Evaluate on validation set
    val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


Epoch [1/10], Validation Loss: 0.08652751230683828, Validation Accuracy: 0.9742731646099142


In [28]:
# After training, you can extract the trained parameters
trained_centers = model.centers.detach().cpu().numpy()
trained_sigmas = model.sigmas.detach().cpu().numpy()

# Print or use the trained parameters
print("Trained Centers:", trained_centers)
print("Trained Sigmas:", trained_sigmas)

Trained Centers: [[ 5.72939974e-02 -4.31551280e-01 -9.85139705e-01 ... -1.22846259e-03
   1.63294273e-06  1.01501844e-09]
 [-4.38452198e-01 -4.18462970e-01 -9.89346378e-01 ... -1.01941469e-04
  -2.80480012e-03  1.43014868e-12]
 [-3.59904254e-01 -3.00404876e-01 -9.32873314e-01 ... -9.18345210e-05
  -8.13778867e-04  1.97773827e-18]
 ...
 [ 5.68983011e-01  3.89639550e-01  1.41794207e+00 ...  1.18695540e-03
   6.26176883e-07  5.45631881e-20]
 [-4.58739444e-01 -4.25028808e-01 -1.01310583e+00 ... -1.00712603e-04
   3.03886764e-06  2.57006704e-11]
 [ 1.52572886e+00  6.01671322e-01  1.31605251e+00 ...  4.33774665e-03
  -3.32060611e-04  6.05219894e-04]]
Trained Sigmas: [4.879355]


In [ ]:
# Save the array to a file
np.save('trained_centers.npy', trained_centers)